In [ ]:
import os.path
import oracledb
import pandas as pd
from ipynb.fs.full.File_Variables import *

In [ ]:
# getting wrigley field data

def get_data():
    
    #initiate oracle client

    oracledb.init_oracle_client(lib_dir = r"C:\ProgramData\Oracle\instantclient_21_10")

    username = "mbhutada"
    userpwd = "Sangeeta@5785"
    host = "ACTRANSITOPS"
    port = 1521
    service_name = "CPC2DS"

    dsn = f'{username}/{userpwd}@{host}:{port}/{service_name}'
    connection = oracledb.connect(dsn)

    cursor = connection.cursor()

    cursor.execute("SELECT ID_MAIN, EVENT, DTE, START_TIME, END_TIME, EVENT_TYPE, VENUE FROM PLANNINGADMIN.SPECEVTS_MAIN WHERE VENUE = :a AND DTE >= :s1 AND DTE <= :s2", (venue, pick_start, pick_end))
    data = cursor.fetchall()

    connection.close()

    df = pd.DataFrame(data, columns = ['ID_MAIN', 'EVENT', 'DTE', 'START_TIME', 'END_TIME', 'EVENT_TYPE', 'VENUE'])
    
    return df

In [ ]:
# getting RTVER_NOs

oracledb.init_oracle_client(lib_dir = r"C:\ProgramData\Oracle\instantclient_21_10")

username = "mbhutada"
userpwd = "Sangeeta@5785"
host = "ACAVAS1"
port = 1521
service_name = "AVAS"

dsn = f'{username}/{userpwd}@{host}:{port}/{service_name}'
connection = oracledb.connect(dsn)

cursor = connection.cursor()

cursor.execute("SELECT RTVER_NO FROM HASTUS2017.RTVER@hasprod WHERE RTVER_ID IN (:a, :b, :c) ORDER BY SCHED_TYPE_ID", (pick_wk, pick_sa, pick_su))
rtver_no = cursor.fetchall()
connection.close()
rtver_df = pd.DataFrame(rtver_no, columns = ['RTVER_NO'])

In [ ]:
# here we are trying to get or update the required pick runtime file

# checking the timestamp for CSV file updates

oracledb.init_oracle_client(lib_dir = r"C:\ProgramData\Oracle\instantclient_21_10")

username = "mbhutada"
userpwd = "Sangeeta@5785"
host = "ACAVAS1"
port = 1521
service_name = "AVAS"

dsn = f'{username}/{userpwd}@{host}:{port}/{service_name}'
connection = oracledb.connect(dsn)

cursor = connection.cursor()


cursor.execute("SELECT RTVER_NO, TO_CHAR(DATETIME_STAMP, 'YYYY-MM-DD') FROM HASTUS2017.RTVER_STAMP@hasprod WHERE RTVER_NO IN (SELECT RTVER_NO FROM HASTUS2017.RTVER@hasprod WHERE RTVER_ID IN (:a, :b, :c))", (pick_wk, pick_sa, pick_su))
timestamp = cursor.fetchall()

connection.close()

timestamp_df = pd.DataFrame(timestamp, columns = ['RTVER_NO', 'TIMESTAMP']) # dataframe to capture unique RTVER_NOs and their latest updation timestamp

path = './timestamp.csv' # csv file to store the unique RTVER_NOs and their latest updation timestamp
check_file = os.path.isfile(path)

if check_file == False:
    
    striker = 0
    timestamp_df.to_csv('./timestamp.csv', index = False)
    
    connection = oracledb.connect(dsn)
    cursor = connection.cursor()
    cursor.execute("SELECT * FROM HASTUS2017.RUNTIME@hasprod WHERE RTVER_NO IN (SELECT RTVER_NO FROM HASTUS2017.RTVER@hasprod WHERE RTVER_ID IN (:a, :b, :c)) AND ROUTE_ID = :d AND RVARIANT_ID IN (:e, :f)", (pick_wk, pick_sa, pick_su, route_id, eb_direction, wb_direction))
    runtimes = cursor.fetchall() 
    connection.close()
    
    runtimes_df = pd.DataFrame(runtimes, columns = ['SYSRECNO', 'RTVER_NO', 'START_PLACE_ID', 'END_PLACE_ID', 'PERIOD_START_TIME', 'PERIOD_END_TIME', 'ROUTE_ID', 'RVARIANT_ID', 'RUN_TIME', 'SYSTXID'])
    csv_name = './runtimes_df_' + pick + '.csv'
    runtimes_df.to_csv(csv_name, index = False)
        
        
else: # if timestamp.csv exists 
    
    striker = 0
    timestamp_df_previous = pd.read_csv('./timestamp.csv')
    
    for i, j in timestamp_df.iterrows():

        if ((timestamp_df_previous['RTVER_NO'] == j['RTVER_NO']) & (timestamp_df_previous['TIMESTAMP'] == j['TIMESTAMP'])).any():
            continue
            
        else:
            connection = oracledb.connect(dsn)
            cursor = connection.cursor()
            cursor.execute("SELECT * FROM HASTUS2017.RUNTIME@hasprod WHERE RTVER_NO IN (SELECT RTVER_NO FROM HASTUS2017.RTVER@hasprod WHERE RTVER_ID IN (:a, :b, :c)) AND ROUTE_ID = :d AND RVARIANT_ID IN (:e, :f)", (pick_wk, pick_sa, pick_su, route_id, eb_direction, wb_direction))
            runtimes = cursor.fetchall() 
            connection.close()
            runtimes_df = pd.DataFrame(runtimes, columns = ['SYSRECNO', 'RTVER_NO', 'START_PLACE_ID', 'END_PLACE_ID', 'PERIOD_START_TIME', 'PERIOD_END_TIME', 'ROUTE_ID', 'RVARIANT_ID', 'RUN_TIME', 'SYSTXID'])
            csv_name = './runtimes_df_' + pick + '.csv'
            runtimes_df.to_csv(csv_name, index = False, mode = 'w')
            striker = 1
            break


if striker == 0:
    pass

else:
    for i, j in zip(timestamp_df['RTVER_NO'].tolist(), timestamp_df['TIMESTAMP'].tolist()):
        if i in timestamp_df_previous['RTVER_NO'].tolist():
            timestamp_df_previous.loc[timestamp_df_previous['RTVER_NO'] == i, 'TIMESTAMP'] = j
            
        else:
            timestamp_df_previous.loc[len(timestamp_df_previous.index)] = [i, j]
            
    timestamp_df_previous.to_csv('./timestamp.csv', index = False, mode = 'w')

In [ ]:
# getting VSCVER_IDs

oracledb.init_oracle_client(lib_dir = r"C:\ProgramData\Oracle\instantclient_21_10")

username = "mbhutada"
userpwd = "Sangeeta@5785"
host = "ACAVAS1"
port = 1521
service_name = "AVAS"

dsn = f'{username}/{userpwd}@{host}:{port}/{service_name}'
connection = oracledb.connect(dsn)

cursor = connection.cursor()

cursor.execute("SELECT VSCVER_ID FROM HASTUS2017.VSCVER@hasprod WHERE BOOKING_ID = :g AND VSC_ID = :h AND SCHED_TYPE_ID IN (:o, :p, :z) ORDER BY SCHED_TYPE_ID", (booking_id, vsc_id, wk_sched_type_id, sa_sched_type_id, su_sched_type_id))
vscver_id = cursor.fetchall()
connection.close()
vscver_df = pd.DataFrame(vscver_id, columns = ['VSCVER_ID'])

In [ ]:
# here we are trying to get or update the required pick vehicle schedule (WB trips) file

# checking the timestamp for CSV file updates

oracledb.init_oracle_client(lib_dir = r"C:\ProgramData\Oracle\instantclient_21_10")

username = "mbhutada"
userpwd = "Sangeeta@5785"
host = "ACAVAS1"
port = 1521
service_name = "AVAS"

dsn = f'{username}/{userpwd}@{host}:{port}/{service_name}'
connection = oracledb.connect(dsn)

cursor = connection.cursor()

cursor.execute("SELECT DISTINCT BOOKING_ID, TO_CHAR(datetime_stamp,'YYYY-MM-DD') FROM HASTUS2017.VSCVER@hasprod WHERE BOOKING_ID = :i AND VSC_ID = :j AND SCHED_TYPE_ID IN (:k, :l, :m)", (booking_id, vsc_id, wk_sched_type_id, sa_sched_type_id, su_sched_type_id))
vsc_timestamp = cursor.fetchall()

connection.close()

vsc_timestamp_df = pd.DataFrame(vsc_timestamp, columns = ['BOOKING_ID', 'TIMESTAMP'])


vsc_path = './vsc_timestamp.csv' 
vsc_check_file = os.path.isfile(vsc_path)

if vsc_check_file == False:
    
    striker = 0
    vsc_timestamp_df.to_csv('./vsc_timestamp.csv', index = False)
    
    connection = oracledb.connect(dsn)
    cursor = connection.cursor()
    cursor.execute("SELECT VSCVER_ID, TRIP_NO, TRIP_ID, START_PLACE, END_PLACE, START_TIME, END_TIME FROM HASTUS2017.TRIP@hasprod WHERE VSCVER_ID IN (SELECT VSCVER_ID FROM HASTUS2017.VSCVER@hasprod WHERE BOOKING_ID = :p AND VSC_ID = :q AND SCHED_TYPE_ID IN (:r, :s, :t)) AND ROUTE_ID = :u AND DIRECTION = :v AND START_PLACE = :w", (booking_id, vsc_id, wk_sched_type_id, sa_sched_type_id, su_sched_type_id, route_id, direction, wb_start_place))
    wb_trips = cursor.fetchall() 
    connection.close()
    
    wb_trips_df = pd.DataFrame(wb_trips, columns = ['VSCVER_ID', 'TRIP_NO', 'TRIP_ID', 'START_PLACE', 'END_PLACE', 'START_TIME', 'END_TIME'])
    csv_name_wb_trips = './wb_trips_' + pick + '.csv'
    wb_trips_df.to_csv(csv_name_wb_trips, index = False)
    

else:  
    
    striker = 0
    vsc_timestamp_df_previous = pd.read_csv('./vsc_timestamp.csv')
    
    for i, j in vsc_timestamp_df.iterrows():

        if ((vsc_timestamp_df_previous['BOOKING_ID'] == j['BOOKING_ID']) & (vsc_timestamp_df_previous['TIMESTAMP'] == j['TIMESTAMP'])).any():
            continue
            
        else:
            connection = oracledb.connect(dsn)
            cursor = connection.cursor()
            cursor.execute("SELECT VSCVER_ID, TRIP_NO, TRIP_ID, START_PLACE, END_PLACE, START_TIME, END_TIME FROM HASTUS2017.TRIP@hasprod WHERE VSCVER_ID IN (SELECT VSCVER_ID FROM HASTUS2017.VSCVER@hasprod WHERE BOOKING_ID = :p AND VSC_ID = :q AND SCHED_TYPE_ID IN (:r, :s, :t)) AND ROUTE_ID = :u AND DIRECTION = :v AND START_PLACE = :w", (booking_id, vsc_id, wk_sched_type_id, sa_sched_type_id, su_sched_type_id, route_id, direction, wb_start_place))
            wb_trips = cursor.fetchall() 
            connection.close()
            wb_trips_df = pd.DataFrame(wb_trips, columns = ['VSCVER_ID', 'TRIP_NO', 'TRIP_ID', 'START_PLACE', 'END_PLACE', 'START_TIME', 'END_TIME'])
            csv_name_wb_trips = './wb_trips_' + pick + '.csv'
            wb_trips_df.to_csv(csv_name_wb_trips, index = False)
            striker = 1
            break

            
if striker == 0:
    pass

else:
    for i, j in zip(vsc_timestamp_df['BOOKING_ID'].tolist(), vsc_timestamp_df['TIMESTAMP'].tolist()):
        if i in vsc_timestamp_df_previous['BOOKING_ID'].tolist():
            vsc_timestamp_df_previous.loc[vsc_timestamp_df_previous['BOOKING_ID'] == i, 'TIMESTAMP'] = j
            
        else:
            vsc_timestamp_df_previous.loc[len(vsc_timestamp_df_previous.index)] = [i, j]
            
    vsc_timestamp_df_previous.to_csv('./vsc_timestamp.csv', index = False, mode = 'w')

In [ ]:
# getting list of holidays

oracledb.init_oracle_client(lib_dir = r"C:\ProgramData\Oracle\instantclient_21_10")

username = "mbhutada"
userpwd = "Sangeeta@5785"
host = "ACAVAS1"
port = 1521
service_name = "AVAS"

dsn = f'{username}/{userpwd}@{host}:{port}/{service_name}'
connection = oracledb.connect(dsn)

cursor = connection.cursor()

cursor.execute("SELECT HOL_DATE FROM HASTUS2017.HOLIDAY@hasprod WHERE HOL_DATE >= :v AND HOL_DATE <= :w", (pick_start, pick_end))
holiday = cursor.fetchall()
connection.close()

holiday_list = pd.DataFrame(holiday, columns = ['HOLIDAY'])['HOLIDAY'].tolist()

def convert_to_date_object(x):
    return x.date()

holiday_list = list(map(convert_to_date_object, holiday_list))